In [ ]:
##
## Summary: In the education industry, high schools prep students for the next chapter in his/her life. 
##    Viewing university statistics and massachusetts high school statistics, 
##    do patterns exist to show patterns between acceptance rates, SAT scores and demographics?
##    
##    The first dataset is:
##        
##        
##        
##    The second dataset is:
##        
##        
        


#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Import the first dataset (Massachusetts high school)
mshs.read_csv('')

#Import the second dataset (University data)
univ.read_json('')

